In [0]:
!pip install transformers

     |████████████████████████████████| 665kB 9.0MB/s 
     |████████████████████████████████| 890kB 11.7MB/s 
     |████████████████████████████████| 1.1MB 43.2MB/s 
     |████████████████████████████████| 3.8MB 59.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=18df090d4408b41940ff90a588850513b474e86b3f50824cd428b674cb686428
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/Shared drives/CS263/data/american_rhetoric/'
import os
os.chdir(root_path + 'speech_bank')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
retval = os.getcwd()
print("Current working directory %s" % retval)

Current working directory /content/gdrive/Shared drives/CS263/data/american_rhetoric/speech_bank


In [0]:
import torch

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [0]:
import pandas as pd

import numpy as np
# !gdown --id 12_VY9LDKd6WBKzNnE_oNsSrjMUR_BTrO
# df = pd.read_csv('parsed-with_summaries_wordfrequency.csv')

!gdown --id 1J7h0H8HsqqcLlbM0zUZGGCn2jh3ZnpgD
df = pd.read_csv('parsed.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1J7h0H8HsqqcLlbM0zUZGGCn2jh3ZnpgD
To: /content/gdrive/Shared drives/CS263/data/american_rhetoric/speech_bank/parsed.csv
16.6MB [00:00, 52.9MB/s]


,title,speaker,transcript,year
0,Congressional Gold Medal Acceptance Address,Aung San Suu Kyi,This is one of the most moving days of my life...,2012
1,Memorial Remarks for Ronald Reagan,Prime Minister Brian Mulroney,"In the spring of 1987, President Reagan and I ...",2004
2,Address to the American Society of Newspaper E...,Dwight D. Eisenhower,"President Bryan, distinguished guests of this ...",1953
3,2004 Democratic National Convention Address,Al Gore,"Thank you, very much. Thank you. Thank you, ve...",2004
4,Speech to the D.C. Federalist Society Lawyers ...,Edwin Meese III,A large part of American history has been the ...,1985


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   title       1202 non-null   object
 1   speaker     1202 non-null   object
 2   transcript  1202 non-null   object
 3   year        1202 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 37.7+ KB


In [0]:
T5_PATH = 't5-large'

t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
t5_model.to(device)
t5_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (

In [0]:
def t5_summarize(i, input_text, num_beams=4, num_words=200):
    preprocess_text = input_text.strip().replace("\n"," ")
    input_text = "summarize: " + preprocess_text
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt", max_length=1024).to(device)
    # summary_task = torch.tensor([[21603, 10]]).to(device)
    # input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)
    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)
    #output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    output = [t5_tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids]
    print('\n\n', i, 'text:', input_text, '\n Summarized:', output)
    return output

In [0]:
summarized  = t5_summarize(1, df['transcript'][1])
print(summarized)




 1 text: summarize: In the spring of 1987, President Reagan and I were driven into a large hangar at the Ottawa Airport, to await the arrival of Mrs. Reagan and my wife, Mila, prior to departure ceremonies for their return to Washington. We were alone except for the security details.President Reagan's visit had been important, demanding, and successful. Our discussions reflected the international agenda of the times: the nuclear threat posed by the Soviet Union and the missile deployment by NATO; pressures in the Warsaw pact; challenges resulting from the Berlin Wall and the ongoing separation of Germany; and bilateral and hemispheric free trade.President Reagan had spoken to Parliament, handled complex files with skill and good humor, strongly impressing his Canadian hosts. And here we were waiting for our wives.When their car drove in a moment later, out stepped Nancy and Mila looking like a million bucks. And as they headed towards us, President Reagan beamed, he threw his arm aro

In [0]:
summarized = ''.join(summarized)
print(len(summarized.split()))
index = summarized.rfind(".")
print(summarized[:index + 1])

43
bob greene: Ronald Reagan was a president who inspired his nation and transformed the world. greene says he embodied the unusual alchemy of history, tradition, achievement. he says Reagan's vision of a united u.s. was based on a sense of the nation's majesty.


In [0]:
for i, text in enumerate(df.transcript):
  summarized = t5_summarize(i, df['transcript'][i])
  # convert list to string
  summarized = ''.join(summarized)
  # find the index of the last occurrence of period. Delete incomplete sentence.
  index1 = summarized.rfind(".")
  index2 = summarized.rfind("?")
  index3 = summarized.rfind('!')
  index = max(index1, index2, index3)

  # removed unfinished sentence in the summary
  df.loc[df.index[i],'summary'] = summarized[:index+1]

  print('\n Number of words in T5 summarized text:', len(summarized.split()), '\n in processed summary', len(df.loc[df.index[i],'summary'].split()))
  print('\n Processed Summary', df.loc[df.index[i],'summary'])

  if i%200 == 0:
    df.to_csv('parsed-with_summaries_t5_11b_200words_raw.csv', index = False)

# print('\n Before removing invalid summaries:', df.info())
# Uncomment to update the parsed-with_summaries data.
# max_length = 200
df.to_csv('parsed-with_summaries_t5_11b_200words_raw.csv', index = False)

df.summary.replace('', np.nan, inplace = True)
# Remove rows with summary NaN

df = df.dropna(subset =['summary']).reset_index(drop = True)

print('After removing invalid summaries:', df.info())
df.to_csv('parsed-with_summaries_t5_11b_200words.csv', index = False)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
df.to_csv('parsed-with_summaries_t5_large_raw.csv', index = False)

In [0]:
df.head()

,title,speaker,transcript,year,summary
0,Congressional Gold Medal Acceptance Address,Aung San Suu Kyi,This is one of the most moving days of my life...,2012,"""this is a moment for which i have been waitin..."
1,Memorial Remarks for Ronald Reagan,Prime Minister Brian Mulroney,"In the spring of 1987, President Reagan and I ...",2004,bob greene: Ronald Reagan was a president who ...
2,Address to the American Society of Newspaper E...,Dwight D. Eisenhower,"President Bryan, distinguished guests of this ...",1953,president george w. bush makes his first forma...
3,2004 Democratic National Convention Address,Al Gore,"Thank you, very much. Thank you. Thank you, ve...",2004,"gloria borger: i didn't run for re-election, b..."
4,Speech to the D.C. Federalist Society Lawyers ...,Edwin Meese III,A large part of American history has been the ...,1985,ruben navarrette: as we approach the bicentenn...


**50 max**
 summarized: ['this is a moment for which I have been waiting for many years. many of you have done so much to uphold our cause that it would take me more than one afternoon to recite all the names of those whom I hold']

 **120 max**
summarized:   summarized: ['this is a moment for which I have been waiting for many years. many of you have done so much to uphold our cause that it would take me more than one afternoon to recite all the names of those whom I hold dear in appreciation and gratitude. this task has been made possible by the reform measures instituted by president U Thein Sein.']
